# Querying Data from Google Earth Engine for a Pandas Dataframe

In [0]:
# To connect to a local runtime:
# https://research.google.com/colaboratory/local-runtimes.html

# Run the following from the command line on your local machine:

# pip install jupyter_http_over_ws
# jupyter serverextension enable --py jupyter_http_over_ws

# jupyter notebook \ --NotebookApp.allow_origin='https://colab.research.google.com' \ --port=8891 \ --NotebookApp.port_retries=0
# jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8891 --NotebookApp.port_retries=0

# If you run into issues connecting with a local kernel, try changing the port number

In [1]:
!pip install earthengine_api

In [2]:
!pip install geopandas

     |████████████████████████████████| 931kB 9.7MB/s 
     |████████████████████████████████| 10.4MB 46.4MB/s 
     |████████████████████████████████| 14.7MB 331kB/s 


In [0]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt

from queue import Empty
import math
import time
import pandas as pd
import geopandas as gpd
from datetime import datetime, timedelta
import ee

In [4]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=j07XNzWUKAKdeJPjgJ4S5TiDi6tUgqPQT9IT_v6mKlo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/zQGysr-hoEpu-CSCtYCVuEqgf7ErsGkkWmajlHrXyAgNWYqv_UxEjS0

Successfully saved authorization token.


## Mounting Drive and Importing Data

In [5]:
# Mount User's Drive
# Copy and paste the passkey from your Google account
# You should use the same account that is operating the Colab file
# Ignore if you aren't accessing this file in Google Colab

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Use os.walk to find the directory with the project folder and create a path object
# May take a while depending on number of sub-folders
# Ensure that the 'GxE with GEE' folder is in 'My Drive', not 'Shared with me'

parent_dir_name = 'GxE with GEE'

for dirpath, subdirs, files in os.walk('/content/drive/My Drive'):
  if parent_dir_name in subdirs:
    parent_path = dirpath + "/" + parent_dir_name

In [7]:
# Set Data Path
## Change the destination to your Drive directory containing the folder 'raw_data.zip'
data_path_end = '/Colab Workspace/Data'
data_path = parent_path + data_path_end
os.chdir(data_path)

# Print the current working directory
os.getcwd()

'/content/drive/My Drive/Research/GxE with GEE/Colab Workspace/Data'

## Importing Latitude and Longitude Data

In [0]:
lat_long = pd.read_pickle('lat_long.pkl')
lat_long.head()

,LAT_COORD,LONG_COORD,COUNTRY,HARVEST_FINISHING_DATE,HARVEST_FINISHING_DAY,HARVEST_FINISHING_MONTH,HARVEST_FINISHING_YEAR,LAT_LONG_DAY_MONTH_YEAR
4ESWYT_19105_254741_5,26.360001,31.40,EGYPT,1983-05-22,22,5,1983,26.36_31.4_22_5_1983
4ESWYT_19207_254741_5,25.000000,23.00,LIBYA,1983-04-30,30,4,1983,25.0_23.0_30_4_1983
4ESWYT_19402_254741_5,36.439999,9.08,TUNISIA,1983-06-08,8,6,1983,36.44_9.08_8_6_1983
4ESWYT_11101_254741_5,-17.410000,30.25,ZIMBABWE,1983-10-27,27,10,1983,-17.41_30.25_27_10_1983
4ESWYT_22001_254741_5,23.590000,90.25,BANGLADESH,1983-03-15,15,3,1983,23.59_90.25_15_3_1983


In [0]:
lat_long.shape

(2759, 8)

## Creating Date Columns

In [0]:
# Drop NA location values
initial_month = 'HARVEST_FINISHING_DATE'

lat_long.dropna(axis=0,how='any',inplace=True,subset=['LAT_COORD'])
lat_long.dropna(axis=0,how='any',inplace=True,subset=['LONG_COORD'])
lat_long.dropna(axis=0,how='any',inplace=True,subset=[initial_month])

lat_long.shape

(3267, 8)

In [0]:
lat_long.head()

,LAT_COORD,LONG_COORD,COUNTRY,HARVEST_FINISHING_DATE,HARVEST_FINISHING_DAY,HARVEST_FINISHING_MONTH,HARVEST_FINISHING_YEAR,LAT_LONG_DAY_MONTH_YEAR
4ESWYT_19105_254741_5,26.360001,31.40,EGYPT,1983-05-22,22,5,1983,26.36_31.4_22_5_1983
4ESWYT_19207_254741_5,25.000000,23.00,LIBYA,1983-04-30,30,4,1983,25.0_23.0_30_4_1983
4ESWYT_19402_254741_5,36.439999,9.08,TUNISIA,1983-06-08,8,6,1983,36.44_9.08_8_6_1983
4ESWYT_11101_254741_5,-17.410000,30.25,ZIMBABWE,1983-10-27,27,10,1983,-17.41_30.25_27_10_1983
4ESWYT_22001_254741_5,23.590000,90.25,BANGLADESH,1983-03-15,15,3,1983,23.59_90.25_15_3_1983


In [0]:
lat_long.drop(columns=(['COUNTRY','HARVEST_FINISHING_DAY','HARVEST_FINISHING_MONTH','HARVEST_FINISHING_YEAR']), inplace=True)

lat_long = lat_long.reset_index()
lat_long['index'] = lat_long['index'].astype(str, errors='ignore')
lat_long.head()

,index,LAT_COORD,LONG_COORD,HARVEST_FINISHING_DATE,LAT_LONG_DAY_MONTH_YEAR
0,4ESWYT_19105_254741_5,26.360001,31.40,1983-05-22,26.36_31.4_22_5_1983
1,4ESWYT_19207_254741_5,25.000000,23.00,1983-04-30,25.0_23.0_30_4_1983
2,4ESWYT_19402_254741_5,36.439999,9.08,1983-06-08,36.44_9.08_8_6_1983
3,4ESWYT_11101_254741_5,-17.410000,30.25,1983-10-27,-17.41_30.25_27_10_1983
4,4ESWYT_22001_254741_5,23.590000,90.25,1983-03-15,23.59_90.25_15_3_1983


In [0]:
from datetime import datetime, timedelta

# Define a function to create a new column one month prior to the initial date column
def make_new_month(month, df, initial_month_column) :
    new_month = pd.Series(df[initial_month_column] - timedelta(days=(30*month)), name=str(initial_month_column + "_" + str(month) + '_MONTHS_PRIOR'))
    return new_month

In [0]:
months_back = 13

for month in range(months_back+1) :
  new_month = initial_month + "_" + str(month) +"_MONTHS_PRIOR"
  lat_long[new_month] = make_new_month(month, df= lat_long, initial_month_column = initial_month)

lat_long.head()

,index,LAT_COORD,LONG_COORD,HARVEST_FINISHING_DATE,LAT_LONG_DAY_MONTH_YEAR,HARVEST_FINISHING_DATE_0_MONTHS_PRIOR,HARVEST_FINISHING_DATE_1_MONTHS_PRIOR,HARVEST_FINISHING_DATE_2_MONTHS_PRIOR,HARVEST_FINISHING_DATE_3_MONTHS_PRIOR,HARVEST_FINISHING_DATE_4_MONTHS_PRIOR,HARVEST_FINISHING_DATE_5_MONTHS_PRIOR,HARVEST_FINISHING_DATE_6_MONTHS_PRIOR,HARVEST_FINISHING_DATE_7_MONTHS_PRIOR,HARVEST_FINISHING_DATE_8_MONTHS_PRIOR,HARVEST_FINISHING_DATE_9_MONTHS_PRIOR,HARVEST_FINISHING_DATE_10_MONTHS_PRIOR,HARVEST_FINISHING_DATE_11_MONTHS_PRIOR,HARVEST_FINISHING_DATE_12_MONTHS_PRIOR,HARVEST_FINISHING_DATE_13_MONTHS_PRIOR
0,4ESWYT_19105_254741_5,26.360001,31.40,1983-05-22,26.36_31.4_22_5_1983,1983-05-22,1983-04-22,1983-03-23,1983-02-21,1983-01-22,1982-12-23,1982-11-23,1982-10-24,1982-09-24,1982-08-25,1982-07-26,1982-06-26,1982-05-27,1982-04-27
1,4ESWYT_19207_254741_5,25.000000,23.00,1983-04-30,25.0_23.0_30_4_1983,1983-04-30,1983-03-31,1983-03-01,1983-01-30,1982-12-31,1982-12-01,1982-11-01,1982-10-02,1982-09-02,1982-08-03,1982-07-04,1982-06-04,1982-05-05,1982-04-05
2,4ESWYT_19402_254741_5,36.439999,9.08,1983-06-08,36.44_9.08_8_6_1983,1983-06-08,1983-05-09,1983-04-09,1983-03-10,1983-02-08,1983-01-09,1982-12-10,1982-11-10,1982-10-11,1982-09-11,1982-08-12,1982-07-13,1982-06-13,1982-05-14
3,4ESWYT_11101_254741_5,-17.410000,30.25,1983-10-27,-17.41_30.25_27_10_1983,1983-10-27,1983-09-27,1983-08-28,1983-07-29,1983-06-29,1983-05-30,1983-04-30,1983-03-31,1983-03-01,1983-01-30,1982-12-31,1982-12-01,1982-11-01,1982-10-02
4,4ESWYT_22001_254741_5,23.590000,90.25,1983-03-15,23.59_90.25_15_3_1983,1983-03-15,1983-02-13,1983-01-14,1982-12-15,1982-11-15,1982-10-16,1982-09-16,1982-08-17,1982-07-18,1982-06-18,1982-05-19,1982-04-19,1982-03-20,1982-02-18


In [0]:
month_cols = lat_long.filter(like=initial_month).columns
lat_long[month_cols] = lat_long[month_cols].astype(str)
lat_long.drop(initial_month, axis=1, inplace = True)

float_32_cols = lat_long.select_dtypes(include=['float32']).columns
lat_long[float_32_cols] = lat_long[float_32_cols].astype('float64')
lat_long.dtypes

index                                      object
LAT_COORD                                 float64
LONG_COORD                                float64
LAT_LONG_DAY_MONTH_YEAR                    object
HARVEST_FINISHING_DATE_0_MONTHS_PRIOR      object
HARVEST_FINISHING_DATE_1_MONTHS_PRIOR      object
HARVEST_FINISHING_DATE_2_MONTHS_PRIOR      object
HARVEST_FINISHING_DATE_3_MONTHS_PRIOR      object
HARVEST_FINISHING_DATE_4_MONTHS_PRIOR      object
HARVEST_FINISHING_DATE_5_MONTHS_PRIOR      object
HARVEST_FINISHING_DATE_6_MONTHS_PRIOR      object
HARVEST_FINISHING_DATE_7_MONTHS_PRIOR      object
HARVEST_FINISHING_DATE_8_MONTHS_PRIOR      object
HARVEST_FINISHING_DATE_9_MONTHS_PRIOR      object
HARVEST_FINISHING_DATE_10_MONTHS_PRIOR     object
HARVEST_FINISHING_DATE_11_MONTHS_PRIOR     object
HARVEST_FINISHING_DATE_12_MONTHS_PRIOR     object
HARVEST_FINISHING_DATE_13_MONTHS_PRIOR     object
dtype: object

In [0]:
lat_long = lat_long.melt(id_vars = ['index', 'LAT_COORD', 'LONG_COORD','LAT_LONG_DAY_MONTH_YEAR'],
                         value_vars = [initial_month + "_" + str(month) + "_MONTHS_PRIOR" for month in range(months_back)],
                       var_name = 'MONTHS_PRIOR', value_name = 'END_DATE')

In [0]:
lat_long['START_DATE'] = (pd.to_datetime(lat_long['END_DATE']) - timedelta(days=30)).astype(str)
lat_long['MONTHS_PRIOR'] = lat_long.MONTHS_PRIOR.str.extract('(\d+)')
lat_long['index_month'] = lat_long['index'] + "_" + lat_long['MONTHS_PRIOR']
lat_long.head()

,index,LAT_COORD,LONG_COORD,LAT_LONG_DAY_MONTH_YEAR,MONTHS_PRIOR,END_DATE,START_DATE,index_month
0,4ESWYT_19105_254741_5,26.360001,31.40,26.36_31.4_22_5_1983,0,1983-05-22 00:00:00.000000000,1983-04-22 00:00:00.000000000,4ESWYT_19105_254741_5_0
1,4ESWYT_19207_254741_5,25.000000,23.00,25.0_23.0_30_4_1983,0,1983-04-30 00:00:00.000000000,1983-03-31 00:00:00.000000000,4ESWYT_19207_254741_5_0
2,4ESWYT_19402_254741_5,36.439999,9.08,36.44_9.08_8_6_1983,0,1983-06-08 00:00:00.000000000,1983-05-09 00:00:00.000000000,4ESWYT_19402_254741_5_0
3,4ESWYT_11101_254741_5,-17.410000,30.25,-17.41_30.25_27_10_1983,0,1983-10-27 00:00:00.000000000,1983-09-27 00:00:00.000000000,4ESWYT_11101_254741_5_0
4,4ESWYT_22001_254741_5,23.590000,90.25,23.59_90.25_15_3_1983,0,1983-03-15 00:00:00.000000000,1983-02-13 00:00:00.000000000,4ESWYT_22001_254741_5_0


In [0]:
lat_long.dtypes

index                       object
LAT_COORD                  float64
LONG_COORD                 float64
LAT_LONG_DAY_MONTH_YEAR     object
MONTHS_PRIOR                object
END_DATE                    object
START_DATE                  object
index_month                 object
dtype: object

In [0]:
lat_long.shape

(35776, 9)

In [0]:
# Manually create a GEE feature collection from a dataframe
# Very slow. More efficient to 
# def df_to_fc(row, df, df_lat_col, df_long_col) :
#   fc_temp_row = ee.FeatureCollection([ee.Feature(ee.Geometry.Point(df[df_lat_col].iloc[row],df[df_long_col].iloc[row]), \
#                                       df.iloc[row].to_dict())])
#   return fc_temp_row

## Convert Dataframe to Shapefile with Geopandas

In [0]:
# https://geopandas.org/gallery/create_geopandas_from_pandas.html

lat_long_gpd = gpd.GeoDataFrame(
    lat_long, geometry=gpd.points_from_xy(lat_long.LONG_COORD, lat_long.LAT_COORD))

In [0]:
# https://automating-gis-processes.github.io/2016/Lesson2-geopandas-basics.html#writing-a-shapefile

start = time.time()

# Create a output path for the data
shape_folder = r"lat_long_shape"

os.mkdir(shape_folder)

shape_file = r"/Lat_Long.shp"

out = shape_folder + shape_file

# Write those rows into a new Shapefile (the default output file format is Shapefile)
lat_long_gpd.to_file(out)

print('Total runtime was (s): {}'.format(time.time() - start))

# Runtime in Colab: 22 seconds

Total runtime was (s): 20.60820722579956


In [0]:
# Zip shapefile for more efficient uploading
# https://thispointer.com/python-how-to-create-a-zip-archive-from-multiple-files-or-directory/

# zip_name = 'lat_long_shape.zip'

# zipObj = ZipFile(zip_name, 'w')
 
# # Add multiple files to the zip
# zipObj.write('Lat_Long.shp')
# zipObj.write('test_1.log')
# zipObj.write('test_2.log')
 
# # close the Zip File
# zipObj.close()

## Upload Shapefile as GEE Asset

In [0]:
# Use the earthengine command line tools
# https://developers.google.com/earth-engine/command_line

# Create folder on the earthengine server
#!earthengine create folder users/theaaronscherf/gxe_gee

In [0]:
# Check user profile to ensure folder is present
#!earthengine ls users/theaaronscherf

In [0]:
#!pip install --upgrade google-cloud-storage

In [0]:
# Authenticate google cloud to access your storage
# You should use the same account that you used to authenticate earth engine
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?code_challenge=BvGoUcTrZI5U1oUosQnmoVxg-m5RLQgw1jGq7bz0HsE&prompt=select_account&code_challenge_method=S256&access_type=offline&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=32555940559.apps.googleusercontent.com&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth


Enter verification code: 4/zAFpOGy7LsIiEJGpA-llBAoVuv8NYyRyw0HByFNnyfq6U09bRFnCOyI

You are now logged in as [theaaronscherf@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


To take a quick anonymous survey, run:
  $ gcloud survey



In [0]:
# Note: You may need to create a project in the Google Cloud 
# https://cloud.google.com/sdk/gcloud/reference/projects/create
!gcloud projects create gxe-gee

Create in progress for [https://cloudresourcemanager.googleapis.com/v1/projects/gxe-gee-three].
Enabling service [cloudapis.googleapis.com] on project [gxe-gee-three]...
Operation "operations/acf.efd42c91-cd9e-4d6c-9333-a8f31d17bd20" finished successfully.


In [0]:
!gcloud config set project gxe-gee

Updated property [core/project].


### Access GCloud Storage from Python

Need to make a bucket in Google Cloud Storage before it can be used. Methods of making buckets from command line with gcloud not working.

In [0]:
# https://cloud.google.com/storage/docs/reference/libraries
# !gcloud iam service-accounts create gxe-gee

In [0]:
# !gcloud projects add-iam-policy-binding gxe-gee --member "serviceAccount:gxe-gee@gxe-gee.iam.gserviceaccount.com" --role "roles/owner"

In [0]:
# !gcloud iam service-accounts keys create gxe_gcloud_key.json --iam-account gxe-gee@gxe-gee.iam.gserviceaccount.com

In [0]:
# !export GOOGLE_APPLICATION_CREDENTIALS= 'gxe_gcloud_key.json'

In [0]:
# Setting application credentials not working properly
# GOOGLE_APPLICATION_CREDENTIALS= '/content/drive/My Drive/Research/GxE with GEE/Colab Workspace/Data/gxe_gcloud_key.json'

In [0]:
# def implicit():
#     from google.cloud import storage

#     # If you don't specify credentials when constructing the client, the
#     # client library will look for credentials in the environment.
#     storage_client = storage.Client()

#     # Make an authenticated API request
#     buckets = list(storage_client.list_buckets())
#     print(buckets)

### Export Shapefile to Specified GCloud Storage Bucket

In [0]:
project_id = 'gxe-gee'
bucket_name = 'gxe-gee-bucket'
!gcloud config set project {project_id}

Updated property [core/project].


In [0]:
shape_folder = r"lat_long_shape"

shape_file = r"/Lat_Long.shp"

out = shape_folder + shape_file

In [0]:
# upload shapefile files to GCloud storage bucket
!gsutil cp  ./lat_long_shape/Lat_Long.shp gs://{bucket_name}/Lat_Long.shp
!gsutil cp  ./lat_long_shape/Lat_Long.cpg gs://{bucket_name}/Lat_Long.cpg
!gsutil cp  ./lat_long_shape/Lat_Long.dbf gs://{bucket_name}/Lat_Long.dbf
!gsutil cp  ./lat_long_shape/Lat_Long.shx gs://{bucket_name}/Lat_Long.shx


Copying file://./lat_long_shape/Lat_Long.shp [Content-Type=application/x-qgis]...
/ [1 files][978.4 KiB/978.4 KiB]                                                
Operation completed over 1 objects/978.4 KiB.                                    
Copying file://./lat_long_shape/Lat_Long.cpg [Content-Type=application/octet-stream]...
/ [1 files][   10.0 B/   10.0 B]                                                
Operation completed over 1 objects/10.0 B.                                       
Copying file://./lat_long_shape/Lat_Long.dbf [Content-Type=application/octet-stream]...
/ [1 files][ 18.1 MiB/ 18.1 MiB]                                                
Operation completed over 1 objects/18.1 MiB.                                     
Copying file://./lat_long_shape/Lat_Long.shx [Content-Type=application/x-qgis]...
/ [1 files][279.6 KiB/279.6 KiB]                                                
Operation completed over 1 objects/279.6 KiB.                                    


## Upload to Cloud Storage

In [0]:
!pip install google.appengine

  Using cached https://files.pythonhosted.org/packages/8d/f1/405f6a85407673cd628fb29fadf84a3dd9f533551a9236609e55e04ad22c/google-appengine-1.5.1.tar.gz
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [0]:
import google.appengine

ModuleNotFoundError: ignored

In [0]:
# Upload shapefile to Earth Enginge server from cloud storage
!earthengine upload table --asset_id=users/theaaronscherf/Lat_Long gs://gxe-gee-bucket/Lat_Long.shp


Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

W0426 18:35:14.368103 140620553484160 __init__.py:46] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_ca

## Function to Query Data from GEE Server and Batch Export Table

In [0]:
feature_coll = ee.FeatureCollection('users/theaaronscherf/Lat_Long')

In [0]:
feature_coll.first().getInfo()

{'geometry': {'coordinates': [-78.49999924483002, -2.499999464715485],
  'type': 'Point'},
 'id': '00000000000000000644',
 'properties': {'END_DATE': '2005-03-08',
  'LAT_COORD': -2.5,
  'LAT_LONG_D': '-2.5_-78.5_6_7_2005',
  'LONG_COORD': -78.5,
  'MONTHS_PRI': '4',
  'START_DATE': '2005-02-06',
  'index': '12SAWYT_53202_279818_66',
  'index_mont': '12SAWYT_53202_279818_66_4'},
 'type': 'Feature'}

In [0]:
ee.Number(feature_coll.size()).getInfo()

35776

In [0]:
def query_GEE_server(image_collection, gee_var, result_fc,
                     buffer = 1000, temp_reducer = ee.Reducer.sum(), 
                     spatial_reducer = ee.Reducer.mean(), scale = 1000, crs = 'EPSG:4326',
                     gee_prefix = 'mean', drive_folder = data_path, fileNamePrefix = 'query_fc_func') :
  
## Select image collection from Earth Engine and select a band  
  imagery = ee.ImageCollection(image_collection).select(gee_var)

## Temporally reduce images for each point to single image per month
  data = ee.ImageCollection(result_fc.map(lambda feature : imagery \
                             .filter(ee.Filter.date(ee.Date(feature.get('START_DATE')),\
                                                    ee.Date(feature.get('END_DATE')))) \
                             .filterBounds(feature.geometry().buffer(buffer)) \
                             .reduce(temp_reducer)\
                             .copyProperties(feature)))

## Reduce regions of images to spatial average per month
## Use an If Algorithm to replace any null values due to dates outside the image collection
  query_feature = ee.FeatureCollection(data.map(lambda image : ee.Feature(ee.Algorithms.If(image.bandNames(), \
                                                         image.reduceRegions( \
                                                              collection = ee.FeatureCollection([ee.Feature( \
                                                                         ee.Geometry.Point([ \
                                                                            image.get('LONG_COORD'),\
                                                                            image.get('LAT_COORD')\
                                                                            ]) \
                                                                         )]), \
                                                              reducer=spatial_reducer, \
                                                              scale=scale, crs = crs),\
                                                         ee.FeatureCollection(\
                                                              ee.Geometry.Point([ \
                                                              image.get('LONG_COORD'),\
                                                              image.get('LAT_COORD')\
                                                              ]))\
                                                            )), \
                         opt_dropNulls=True)).flatten()

# Batch export the task to Earth Engine and send the results to Drive
  mytask = ee.batch.Export.table.toDrive(collection=query_feature, \
#                                              selectors = [gee_prefix], \
#                                              folder = drive_folder, \
                                              fileNamePrefix = fileNamePrefix, \
                                              fileFormat='CSV')

  mytask.start()

In [0]:
def query_GEE_id(image_collection, gee_var, result_fc,
                     buffer = 1000, temp_reducer = ee.Reducer.sum(), 
                     fileNamePrefix = 'query_fc_func') :
  
## Select image collection from Earth Engine and select a band  
  imagery = ee.ImageCollection(image_collection).select(gee_var)

## Temporally reduce images for each point to single image per month
  data = ee.ImageCollection(result_fc.map(lambda feature : imagery \
                             .filter(ee.Filter.date(ee.Date(feature.get('START_DATE')),\
                                                    ee.Date(feature.get('END_DATE')))) \
                             .filterBounds(feature.geometry().buffer(buffer)) \
                             .reduce(temp_reducer)\
                             .copyProperties(feature)))

## Reduce regions of images to spatial average per month
## Use an If Algorithm to replace any null values due to dates outside the image collection
  query_feature = ee.FeatureCollection(data.map(lambda image : image))
## Use an If Algorithm to replace any null values due to dates outside the image collection
  query_feature = ee.FeatureCollection(data.map(lambda image : ee.Feature(ee.Algorithms.If(image.bandNames(), \
                                                         image.reduceRegions( \
                                                              collection = ee.FeatureCollection([ee.Feature( \
                                                                         ee.Geometry.Point([ \
                                                                            image.get('LONG_COORD'),\
                                                                            image.get('LAT_COORD')\
                                                                            ]) \
                                                                         )]), \
                                                              reducer=spatial_reducer, \
                                                              scale=scale, crs = crs),\
                                                         ee.FeatureCollection(\
                                                              ee.Geometry.Point([ \
                                                              image.get('LONG_COORD'),\
                                                              image.get('LAT_COORD')\
                                                              ]))\
                                                            )), \
                         opt_dropNulls=True)).flatten()

# Batch export the task to Earth Engine and send the results to Drive
  mytask = ee.batch.Export.table.toDrive(collection=query_feature, \
#                                              selectors = [gee_prefix], \
#                                              folder = drive_folder, \
                                              fileNamePrefix = fileNamePrefix, \
                                              fileFormat='CSV')

  mytask.start()

In [0]:
def query_GEE_fix_date(image_collection, gee_var, result_fc,
                     buffer = 1000, 
                     spatial_reducer = ee.Reducer.mean(), scale = 1000, crs = 'EPSG:4326',
                     gee_prefix = 'mean', drive_folder = data_path, fileNamePrefix = 'query_fc_func') :
  
  image = ee.Image(image_collection).select(gee_var)
## Temporally reduce images for each point to single image per month
  data = ee.ImageCollection(result_fc.map(lambda feature : image.copyProperties(feature)))


## Use an If Algorithm to replace any null values due to dates outside the image collection
  query_feature = ee.FeatureCollection(data.map(lambda image : ee.Feature(ee.Algorithms.If(image.bandNames(), \
                                                         image.reduceRegions( \
                                                              collection = ee.FeatureCollection([ee.Feature( \
                                                                         ee.Geometry.Point([ \
                                                                            image.get('LONG_COORD'),\
                                                                            image.get('LAT_COORD')\
                                                                            ]) \
                                                                         )]), \
                                                              reducer=spatial_reducer, \
                                                              scale=scale, crs = crs),\
                                                         ee.FeatureCollection(\
                                                              ee.Geometry.Point([ \
                                                              image.get('LONG_COORD'),\
                                                              image.get('LAT_COORD')\
                                                              ]))\
                                                            )), \
                         opt_dropNulls=True)).flatten()

# Batch export the task to Earth Engine and send the results to Drive
  mytask = ee.batch.Export.table.toDrive(collection=query_feature, \
                                              fileNamePrefix = fileNamePrefix, \
                                              fileFormat='CSV')

  mytask.start()

In [0]:
# Query an unreduced data set, just to get the ID properties associated with the query
query_GEE_id(image_collection = 'ECMWF/ERA5/DAILY', gee_var = 'maximum_2m_air_temperature', 
                 result_fc = feature_coll, fileNamePrefix = 'gee_id_values', 
                 temp_reducer = ee.Reducer.mean())

In [0]:
# Query the altitude for each point
query_GEE_fix_date(image_collection = "USGS/SRTMGL1_003", gee_var = 'elevation', 
                 result_fc = feature_coll, fileNamePrefix = 'query_gee_av_altitude_usgs',
                 spatial_reducer = ee.Reducer.mean())

## Call Query Function over All Datasets and Variables

In [8]:
input_choices = pd.DataFrame([['ECMWF/ERA5/DAILY','total_precipitation',ee.Reducer.sum(), ee.Reducer.mean(), 'query_era5_total_precip'],
                              ['ECMWF/ERA5/DAILY','mean_2m_air_temperature',ee.Reducer.mean(), ee.Reducer.mean(), 'query_era5_av_mean_air_temp'],
                              ['ECMWF/ERA5/DAILY','maximum_2m_air_temperature',ee.Reducer.mean(), ee.Reducer.mean(),'query_era5_av_max_air_temp'],
                              ['ECMWF/ERA5/DAILY','minimum_2m_air_temperature',ee.Reducer.mean(), ee.Reducer.mean(),'query_era5_av_min_air_temp'],
                              ["NASA/FLDAS/NOAH01/C/GL/M/V001",'Evap_tavg',ee.Reducer.sum(), ee.Reducer.mean(),'query_fldas_total_evap'],
                              ["NASA/FLDAS/NOAH01/C/GL/M/V001",'Psurf_f_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_surf_press'],
                              ["NASA/FLDAS/NOAH01/C/GL/M/V001",'Qair_f_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_spec_humid'],
                              ["NASA/FLDAS/NOAH01/C/GL/M/V001",'Qg_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_soil_heatflux'],
                              ["NASA/FLDAS/NOAH01/C/GL/M/V001",'Qsb_tavg',ee.Reducer.sum(), ee.Reducer.mean(),'query_fldas_total_ground_runoff'],
                              ["NASA/FLDAS/NOAH01/C/GL/M/V001",'RadT_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_surf_radtemp'],
                             ["NASA/FLDAS/NOAH01/C/GL/M/V001",'Rainf_f_tavg',ee.Reducer.sum(), ee.Reducer.mean(),'query_fldas_total_precip_rate'],
                             ["NASA/FLDAS/NOAH01/C/GL/M/V001",'SoilMoi00_10cm_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_soil_moist_0_10'],
                             ["NASA/FLDAS/NOAH01/C/GL/M/V001",'SoilMoi10_40cm_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_soil_moist_10_40'],
                             ["NASA/FLDAS/NOAH01/C/GL/M/V001",'SoilMoi100_200cm_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_soil_moist_100_200'],
                             ["NASA/FLDAS/NOAH01/C/GL/M/V001",'SoilMoi40_100cm_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_soil_moist_40_100'],
                             ["NASA/FLDAS/NOAH01/C/GL/M/V001",'SoilTemp00_10cm_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_soil_temp_0_10'],
                             ["NASA/FLDAS/NOAH01/C/GL/M/V001",'SoilTemp10_40cm_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_soil_temp_10_40'],
                             ["NASA/FLDAS/NOAH01/C/GL/M/V001",'SoilTemp100_200cm_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_soil_temp_100_200'],
                             ["NASA/FLDAS/NOAH01/C/GL/M/V001",'SoilTemp40_100cm_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_soil_temp_40_100'],
                             ["NASA/FLDAS/NOAH01/C/GL/M/V001",'Tair_f_tavg',ee.Reducer.mean(), ee.Reducer.mean(),'query_fldas_av_air_temp'],
                             ["IDAHO_EPSCOR/TERRACLIMATE","aet",ee.Reducer.sum(), ee.Reducer.mean(), 'query_terra_sum_evap'],
                             ["IDAHO_EPSCOR/TERRACLIMATE","def",ee.Reducer.sum(), ee.Reducer.mean(), 'query_terra_sum_def'],
                             ["IDAHO_EPSCOR/TERRACLIMATE","soil",ee.Reducer.mean(), ee.Reducer.mean(), 'query_terra_av_soil_moist'],
                             ["IDAHO_EPSCOR/TERRACLIMATE","vap",ee.Reducer.mean(), ee.Reducer.mean(), 'query_terra_av_vap_press'],
                             ["IDAHO_EPSCOR/TERRACLIMATE","tmmn",ee.Reducer.sum(), ee.Reducer.mean(), 'query_terra_av_min_temp'],
                             ["IDAHO_EPSCOR/TERRACLIMATE","tmmx",ee.Reducer.sum(), ee.Reducer.mean(), 'query_terra_av_max_temp']
                              ],
                             columns=['image_collection','gee_var','temp_reducer','spatial_reducer','fileNamePrefix'])
input_choices

,image_collection,gee_var,temp_reducer,spatial_reducer,fileNamePrefix
0,ECMWF/ERA5/DAILY,total_precipitation,"ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...","ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...",query_era5_total_precip
1,ECMWF/ERA5/DAILY,mean_2m_air_temperature,"ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...","ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...",query_era5_av_mean_air_temp
2,ECMWF/ERA5/DAILY,maximum_2m_air_temperature,"ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...","ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...",query_era5_av_max_air_temp
3,ECMWF/ERA5/DAILY,minimum_2m_air_temperature,"ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...","ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...",query_era5_av_min_air_temp
4,NASA/FLDAS/NOAH01/C/GL/M/V001,Evap_tavg,"ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...","ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...",query_fldas_total_evap
5,NASA/FLDAS/NOAH01/C/GL/M/V001,Psurf_f_tavg,"ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...","ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...",query_fldas_av_surf_press
6,NASA/FLDAS/NOAH01/C/GL/M/V001,Qair_f_tavg,"ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...","ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...",query_fldas_av_spec_humid
7,NASA/FLDAS/NOAH01/C/GL/M/V001,Qg_tavg,"ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...","ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...",query_fldas_av_soil_heatflux
8,NASA/FLDAS/NOAH01/C/GL/M/V001,Qsb_tavg,"ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...","ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...",query_fldas_total_ground_runoff
9,NASA/FLDAS/NOAH01/C/GL/M/V001,RadT_tavg,"ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...","ee.Reducer({\n ""type"": ""Invocation"",\n ""argu...",query_fldas_av_surf_radtemp


In [0]:
for row in range(input_choices.shape[0]) :
  query_GEE_server(image_collection = input_choices['image_collection'][row],
                   gee_var = input_choices['gee_var'][row],
                   temp_reducer = input_choices['temp_reducer'][row],
                   spatial_reducer= input_choices['spatial_reducer'][row],
                   fileNamePrefix = input_choices['fileNamePrefix'][row],
                   result_fc = feature_coll)

In [0]:
# To check which tasks you currently having processing in GEE:
# Look at Tasks in the Code Editor (https://code.earthengine.google.com/) or the following command:
!earthengine task list

## Importing GEE Data from Drive and Combining with Lat-Long Data using ID Values

In [0]:
os.mkdir('GEE_Data')

In [0]:
os.chdir('/content/drive/My Drive')

In [0]:
import shutil
# Move all data CSV files from directory source to data path folder
# Don't run until all the GEE tasks have finished importing data
# The time it takes depends on the number of variables queried and size of the input data
# Querying two variables for ~3,500 rows each took about 4 minutes
# 20 variables for the same size data took 2 hours
for row in range(input_choices.shape[0]):
  source = input_choices['fileNamePrefix'][row] + '.csv'
  dest = data_path + '/GEE_Data'
  shutil.move(src = source, dst = dest)

In [0]:
shutil.move(src = 'gee_id_values.csv', dst= data_path)

In [9]:
os.chdir(data_path + '/GEE_Data')
os.getcwd()

'/content/drive/My Drive/Research/GxE with GEE/Colab Workspace/Data/GEE_Data'

In [10]:
gee_data = pd.read_csv('gee_id_values.csv')
gee_data.drop(columns=['system:bands','system:band_names','.geo'], inplace=True)
gee_data.head()

,system:index,END_DATE,LAT_COORD,LAT_LONG_D,LONG_COORD,MONTHS_PRI,START_DATE,index,index_mont
0,00000000000000000644,2005-03-08,-2.5,-2.5_-78.5_6_7_2005,-78.5,4,2005-02-06,12SAWYT_53202_279818_66,12SAWYT_53202_279818_66_4
1,00000000000000001104,2005-02-06,-2.5,-2.5_-78.5_6_7_2005,-78.5,5,2005-01-07,12SAWYT_53202_279818_66,12SAWYT_53202_279818_66_5
2,00000000000000001bc4,2005-01-07,-2.5,-2.5_-78.5_6_7_2005,-78.5,6,2004-12-08,12SAWYT_53202_279818_66,12SAWYT_53202_279818_66_6
3,00000000000000002684,2004-12-08,-2.5,-2.5_-78.5_6_7_2005,-78.5,7,2004-11-08,12SAWYT_53202_279818_66,12SAWYT_53202_279818_66_7
4,00000000000000003941,2005-07-06,-2.5,-2.5_-78.5_6_7_2005,-78.5,0,2005-06-06,12SAWYT_53202_279818_66,12SAWYT_53202_279818_66_0


In [0]:
for row in range(input_choices.shape[0]) :
  file_name = input_choices['fileNamePrefix'][row] + '.csv'
  temp = pd.read_csv(file_name)
  temp['system:index'] = temp['system:index'].str.split("_").str[0]
  temp.drop(columns=['.geo'], inplace=True)
  gee_data = gee_data.merge(temp, on='system:index')
  gee_data.rename(columns={'mean':input_choices['gee_var'][row]},inplace=True)

In [0]:
gee_alt = pd.read_csv('query_gee_av_altitude_usgs.csv')
gee_alt['system:index'] = gee_alt['system:index'].str.split("_").str[0]
gee_alt.drop(columns=['.geo'],inplace=True)
gee_data = gee_data.merge(gee_alt, on='system:index')
gee_data.rename(columns={'mean':'av_altitude'},inplace=True)

In [13]:
gee_data.tail()

,system:index,END_DATE,LAT_COORD,LAT_LONG_D,LONG_COORD,MONTHS_PRI,START_DATE,index,index_mont,total_precipitation,mean_2m_air_temperature,maximum_2m_air_temperature,minimum_2m_air_temperature,Evap_tavg,Psurf_f_tavg,Qair_f_tavg,Qg_tavg,Qsb_tavg,RadT_tavg,Rainf_f_tavg,SoilMoi00_10cm_tavg,SoilMoi10_40cm_tavg,SoilMoi100_200cm_tavg,SoilMoi40_100cm_tavg,SoilTemp00_10cm_tavg,SoilTemp10_40cm_tavg,SoilTemp100_200cm_tavg,SoilTemp40_100cm_tavg,Tair_f_tavg,aet,def,soil,vap,tmmn,tmmx,av_altitude
35771,00000000000000005d1b,1998-07-07,-3.22,-3.22_36.4_4_11_1998,36.400002,4,1998-06-07,6HRWYT_12812_61665_1,6HRWYT_12812_61665_1_4,0.011513,291.944092,298.102051,286.691833,0.000019,81333.015625,0.007788,0.069640,3.358808e-08,289.280212,0.000003,0.217187,0.333581,0.345543,0.338192,289.279510,289.332153,289.486816,289.414612,287.288483,68.0,822.0,155.0,1354.0,108.0,199.0,1790.0
35772,00000000000000006405,1998-02-07,-3.22,-3.22_36.4_4_11_1998,36.400002,9,1998-01-08,6HRWYT_12812_61665_1,6HRWYT_12812_61665_1_9,0.108631,295.414612,300.011658,291.410583,0.000051,81204.429688,0.010635,0.247563,5.559194e-06,290.481384,0.000065,0.399805,0.403852,0.417589,0.408991,290.455780,290.341797,290.144318,290.269104,291.354675,1020.0,0.0,615.0,1626.0,144.0,255.0,1790.0
35773,00000000000000006ec5,1998-01-08,-3.22,-3.22_36.4_4_11_1998,36.400002,10,1997-12-09,6HRWYT_12812_61665_1,6HRWYT_12812_61665_1_10,0.228356,295.194672,299.688263,291.350433,0.000051,81067.281250,0.011193,-0.526289,3.733947e-06,290.141846,0.000071,0.395160,0.398108,0.410254,0.402164,290.190094,290.361267,290.472168,290.464203,290.582001,982.0,0.0,615.0,1624.0,152.0,241.0,1790.0
35774,00000000000000007985,1997-12-09,-3.22,-3.22_36.4_4_11_1998,36.400002,11,1997-11-09,6HRWYT_12812_61665_1,6HRWYT_12812_61665_1_11,0.303362,295.238312,299.585083,291.754303,0.000048,81100.187500,0.010942,-0.357006,1.354501e-06,290.696838,0.000127,0.380373,0.388552,0.366587,0.387259,290.721222,290.839325,290.959656,290.941376,290.645813,939.0,0.0,615.0,1616.0,141.0,240.0,1790.0
35775,00000000000000008445,1997-11-09,-3.22,-3.22_36.4_4_11_1998,36.400002,12,1997-10-10,6HRWYT_12812_61665_1,6HRWYT_12812_61665_1_12,0.233303,295.852448,301.676178,291.374359,0.000031,81070.726562,0.010556,-0.062820,2.926824e-10,291.868835,0.000082,0.306478,0.328652,0.279729,0.305550,291.861755,291.834839,291.290497,291.731232,290.537933,849.0,0.0,615.0,1604.0,142.0,225.0,1790.0


In [14]:
gee_data.shape

(35776, 36)

In [15]:
value_cols = list(gee_data.columns.values)
to_remove = {'system:index','LAT_LONG_D','MONTHS_PRI','index','index_mont'}

value_cols = [col for col in value_cols if col not in to_remove]
value_cols

['END_DATE',
 'LAT_COORD',
 'LONG_COORD',
 'START_DATE',
 'total_precipitation',
 'mean_2m_air_temperature',
 'maximum_2m_air_temperature',
 'minimum_2m_air_temperature',
 'Evap_tavg',
 'Psurf_f_tavg',
 'Qair_f_tavg',
 'Qg_tavg',
 'Qsb_tavg',
 'RadT_tavg',
 'Rainf_f_tavg',
 'SoilMoi00_10cm_tavg',
 'SoilMoi10_40cm_tavg',
 'SoilMoi100_200cm_tavg',
 'SoilMoi40_100cm_tavg',
 'SoilTemp00_10cm_tavg',
 'SoilTemp10_40cm_tavg',
 'SoilTemp100_200cm_tavg',
 'SoilTemp40_100cm_tavg',
 'Tair_f_tavg',
 'aet',
 'def',
 'soil',
 'vap',
 'tmmn',
 'tmmx',
 'av_altitude']

In [16]:
unmelted_gee_data = gee_data.pivot(index='LAT_LONG_D', columns='MONTHS_PRI', values=value_cols)
unmelted_gee_data.head()

END_DATE                          ... av_altitude            
MONTHS_PRI                      0           1           2   ...          10    11    12
LAT_LONG_D                                                  ...                        
-0.16_29.1_10_9_1997    1997-09-10  1997-08-11  1997-07-12  ...        1959  1959  1959
-0.16_29.1_23_8_1994    1994-08-23  1994-07-24  1994-06-24  ...        1959  1959  1959
-0.16_29.1_7_8_1996     1996-08-07  1996-07-08  1996-06-08  ...        1959  1959  1959
-0.22_-78.32_12_8_2011  2011-08-12  2011-07-13  2011-06-13  ...        2756  2756  2756
-0.22_-78.32_16_7_2010  2010-07-16  2010-06-16  2010-05-17  ...        2756  2756  2756

[5 rows x 403 columns]

In [0]:
unmelted_gee_data.columns = unmelted_gee_data.columns.to_flat_index()

In [18]:
unmelted_gee_data.head()

,"(END_DATE, 0)","(END_DATE, 1)","(END_DATE, 2)","(END_DATE, 3)","(END_DATE, 4)","(END_DATE, 5)","(END_DATE, 6)","(END_DATE, 7)","(END_DATE, 8)","(END_DATE, 9)","(END_DATE, 10)","(END_DATE, 11)","(END_DATE, 12)","(LAT_COORD, 0)","(LAT_COORD, 1)","(LAT_COORD, 2)","(LAT_COORD, 3)","(LAT_COORD, 4)","(LAT_COORD, 5)","(LAT_COORD, 6)","(LAT_COORD, 7)","(LAT_COORD, 8)","(LAT_COORD, 9)","(LAT_COORD, 10)","(LAT_COORD, 11)","(LAT_COORD, 12)","(LONG_COORD, 0)","(LONG_COORD, 1)","(LONG_COORD, 2)","(LONG_COORD, 3)","(LONG_COORD, 4)","(LONG_COORD, 5)","(LONG_COORD, 6)","(LONG_COORD, 7)","(LONG_COORD, 8)","(LONG_COORD, 9)","(LONG_COORD, 10)","(LONG_COORD, 11)","(LONG_COORD, 12)","(START_DATE, 0)",...,"(vap, 12)","(tmmn, 0)","(tmmn, 1)","(tmmn, 2)","(tmmn, 3)","(tmmn, 4)","(tmmn, 5)","(tmmn, 6)","(tmmn, 7)","(tmmn, 8)","(tmmn, 9)","(tmmn, 10)","(tmmn, 11)","(tmmn, 12)","(tmmx, 0)","(tmmx, 1)","(tmmx, 2)","(tmmx, 3)","(tmmx, 4)","(tmmx, 5)","(tmmx, 6)","(tmmx, 7)","(tmmx, 8)","(tmmx, 9)","(tmmx, 10)","(tmmx, 11)","(tmmx, 12)","(av_altitude, 0)","(av_altitude, 1)","(av_altitude, 2)","(av_altitude, 3)","(av_altitude, 4)","(av_altitude, 5)","(av_altitude, 6)","(av_altitude, 7)","(av_altitude, 8)","(av_altitude, 9)","(av_altitude, 10)","(av_altitude, 11)","(av_altitude, 12)"
LAT_LONG_D,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-0.16_29.1_10_9_1997,1997-09-10,1997-08-11,1997-07-12,1997-06-12,1997-05-13,1997-04-13,1997-03-14,1997-02-12,1997-01-13,1996-12-14,1996-11-14,1996-10-15,1996-09-15,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,1997-08-11,...,1677,126,104,109,110,129,121,117,118,119,123,104,119,109,247,240,235,234,238,237,253,245,252,251,235,230,241,1959,1959,1959,1959,1959,1959,1959,1959,1959,1959,1959,1959,1959
-0.16_29.1_23_8_1994,1994-08-23,1994-07-24,1994-06-24,1994-05-25,1994-04-25,1994-03-26,1994-02-24,1994-01-25,1993-12-26,1993-11-26,1993-10-27,1993-09-27,1993-08-28,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,1994-07-24,...,1941,104,95,109,119,106,103,106,102,118,133,99,105,123,234,219,230,237,240,235,233,233,234,265,230,229,244,1959,1959,1959,1959,1959,1959,1959,1959,1959,1959,1959,1959,1959
-0.16_29.1_7_8_1996,1996-08-07,1996-07-08,1996-06-08,1996-05-09,1996-04-09,1996-03-10,1996-02-09,1996-01-10,1995-12-11,1995-11-11,1995-10-12,1995-09-12,1995-08-13,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,1996-07-08,...,1941,104,107,109,105,114,106,116,112,115,119,118,116,131,240,232,236,245,243,239,255,250,244,254,243,238,237,1959,1959,1959,1959,1959,1959,1959,1959,1959,1959,1959,1959,1959
-0.22_-78.32_12_8_2011,2011-08-12,2011-07-13,2011-06-13,2011-05-14,2011-04-14,2011-03-15,2011-02-13,2011-01-14,2010-12-15,2010-11-15,2010-10-16,2010-09-16,2010-08-17,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,2011-07-13,...,1094,70,72,79,79,76,73,71,66,66,70,71,67,73,206,192,192,191,185,177,179,189,180,179,191,192,197,2756,2756,2756,2756,2756,2756,2756,2756,2756,2756,2756,2756,2756
-0.22_-78.32_16_7_2010,2010-07-16,2010-06-16,2010-05-17,2010-04-17,2010-03-18,2010-02-16,2010-01-17,2009-12-18,2009-11-18,2009-10-19,2009-09-19,2009-08-20,2009-07-21,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,2010-06-16,...,1202,71,73,83,90,84,72,71,72,72,71,76,80,71,193,187,193,184,192,203,208,203,197,204,208,210,191,2756,2756,2756,2756,2756,2756,2756,2756,2756,2756,2756,2756,2756


In [0]:
unmelted_gee_data.drop(columns=[('av_altitude', x) for x in range(1,13)],inplace=True)

In [20]:
unmelted_gee_data.head()

,"(END_DATE, 0)","(END_DATE, 1)","(END_DATE, 2)","(END_DATE, 3)","(END_DATE, 4)","(END_DATE, 5)","(END_DATE, 6)","(END_DATE, 7)","(END_DATE, 8)","(END_DATE, 9)","(END_DATE, 10)","(END_DATE, 11)","(END_DATE, 12)","(LAT_COORD, 0)","(LAT_COORD, 1)","(LAT_COORD, 2)","(LAT_COORD, 3)","(LAT_COORD, 4)","(LAT_COORD, 5)","(LAT_COORD, 6)","(LAT_COORD, 7)","(LAT_COORD, 8)","(LAT_COORD, 9)","(LAT_COORD, 10)","(LAT_COORD, 11)","(LAT_COORD, 12)","(LONG_COORD, 0)","(LONG_COORD, 1)","(LONG_COORD, 2)","(LONG_COORD, 3)","(LONG_COORD, 4)","(LONG_COORD, 5)","(LONG_COORD, 6)","(LONG_COORD, 7)","(LONG_COORD, 8)","(LONG_COORD, 9)","(LONG_COORD, 10)","(LONG_COORD, 11)","(LONG_COORD, 12)","(START_DATE, 0)",...,"(vap, 0)","(vap, 1)","(vap, 2)","(vap, 3)","(vap, 4)","(vap, 5)","(vap, 6)","(vap, 7)","(vap, 8)","(vap, 9)","(vap, 10)","(vap, 11)","(vap, 12)","(tmmn, 0)","(tmmn, 1)","(tmmn, 2)","(tmmn, 3)","(tmmn, 4)","(tmmn, 5)","(tmmn, 6)","(tmmn, 7)","(tmmn, 8)","(tmmn, 9)","(tmmn, 10)","(tmmn, 11)","(tmmn, 12)","(tmmx, 0)","(tmmx, 1)","(tmmx, 2)","(tmmx, 3)","(tmmx, 4)","(tmmx, 5)","(tmmx, 6)","(tmmx, 7)","(tmmx, 8)","(tmmx, 9)","(tmmx, 10)","(tmmx, 11)","(tmmx, 12)","(av_altitude, 0)"
LAT_LONG_D,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-0.16_29.1_10_9_1997,1997-09-10,1997-08-11,1997-07-12,1997-06-12,1997-05-13,1997-04-13,1997-03-14,1997-02-12,1997-01-13,1996-12-14,1996-11-14,1996-10-15,1996-09-15,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,1997-08-11,...,1700,1655,1600,1667,1640,1654,1589,1358,1513,1621,1678,1686,1677,126,104,109,110,129,121,117,118,119,123,104,119,109,247,240,235,234,238,237,253,245,252,251,235,230,241,1959
-0.16_29.1_23_8_1994,1994-08-23,1994-07-24,1994-06-24,1994-05-25,1994-04-25,1994-03-26,1994-02-24,1994-01-25,1993-12-26,1993-11-26,1993-10-27,1993-09-27,1993-08-28,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,1994-07-24,...,1682,1409,1671,1723,1670,1617,1496,1578,1698,2164,1514,1583,1941,104,95,109,119,106,103,106,102,118,133,99,105,123,234,219,230,237,240,235,233,233,234,265,230,229,244,1959
-0.16_29.1_7_8_1996,1996-08-07,1996-07-08,1996-06-08,1996-05-09,1996-04-09,1996-03-10,1996-02-09,1996-01-10,1995-12-11,1995-11-11,1995-10-12,1995-09-12,1995-08-13,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,-0.16,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,29.1,1996-07-08,...,1642,1683,1637,1634,1704,1508,1572,1619,1636,1929,1706,1748,1941,104,107,109,105,114,106,116,112,115,119,118,116,131,240,232,236,245,243,239,255,250,244,254,243,238,237,1959
-0.22_-78.32_12_8_2011,2011-08-12,2011-07-13,2011-06-13,2011-05-14,2011-04-14,2011-03-15,2011-02-13,2011-01-14,2010-12-15,2010-11-15,2010-10-16,2010-09-16,2010-08-17,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,2011-07-13,...,1118,1061,1209,1219,1138,1133,1124,1131,1095,1138,1185,1139,1094,70,72,79,79,76,73,71,66,66,70,71,67,73,206,192,192,191,185,177,179,189,180,179,191,192,197,2756
-0.22_-78.32_16_7_2010,2010-07-16,2010-06-16,2010-05-17,2010-04-17,2010-03-18,2010-02-16,2010-01-17,2009-12-18,2009-11-18,2009-10-19,2009-09-19,2009-08-20,2009-07-21,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-0.22,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,-78.32,2010-06-16,...,1074,1137,1264,1312,1315,1297,1283,1292,1261,1256,1207,1223,1202,71,73,83,90,84,72,71,72,72,71,76,80,71,193,187,193,184,192,203,208,203,197,204,208,210,191,2756


## Exporting Resulting Environmental Data

In [0]:
unmelted_gee_data.to_pickle('earth_engine_data.pkl')